# Feature study - Pombourg scene

This notebook details the geometric features for the `Pombourg` dataset. In order to illustrate the study, the screencast below represents the point cloud as seen with `CloudCompare`:

<img src="../../docs/images/pombourg.png" width="300px"></img>

## Imports

In [ ]:
%matplotlib inline

In [ ]:
from pathlib import Path

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
plt.style.use("seaborn-notebook")

In [ ]:
from geo3dfeatures.io import las as read_las
from geo3dfeatures.extract import process_full
from geo3dfeatures.extract import compute_tree, request_tree, extract

## Data preparation

In [ ]:
PREFIX_NAME = "Pombourg"
SCENE_FILE = PREFIX_NAME + ".las"
EBOULIS_FILE = PREFIX_NAME + "_eboulis.las"
VEGETATION_FILE = PREFIX_NAME + "_vegetation.las"
FALAISE_FILE = PREFIX_NAME + "_falaise.las"

In [ ]:
DATAFOLDER = "data"

In [ ]:
DATADIR = Path("../..", DATAFOLDER)

In [ ]:
LABELS = {"eboulis": 0,
          "falaise": 1,
          "vegetation": 2}

In [ ]:
# Coordinates labels
COORDS = list("xyz")
# RGB color labels
COULEURS = list("rgb")

In [ ]:
rawdata = read_las(DATADIR / "input" / SCENE_FILE)
rawdata = pd.DataFrame(rawdata, columns=list("xyzrgb"))

In [ ]:
print("dimension: ", rawdata.shape)
rawdata.head()

## Spatial index

In [ ]:
LEAF_SIZE = 1_000
N_NEIGHBORS = 10

In [ ]:
tree = compute_tree(rawdata[COORDS].values, leaf_size=LEAF_SIZE)

We request the KD-tree so as to retrieve the neighbors (k=10) of a given point, and the corresponding distances to it.

In [ ]:
random_point = rawdata[COORDS].sample()
distance, neighbors_index = request_tree(random_point.values, tree, N_NEIGHBORS)
distance, neighbors_index = distance.squeeze(), neighbors_index.squeeze()
print("distance: ", distance)
print("index: ", neighbors_index)

Hence the following table contains the requested point (first row) and its 10 closest neighbors.

In [ ]:
rawdata.loc[neighbors_index]

We can compute some basic statistics about distance with neighbors (expressed in meters).

In [ ]:
pd.Series(distance).describe()

## Class sample features

We aim at computing all the features on every `Pombourg` sample, with respect to the entire scene. The subsequent geometric features can be computed considering a set of different neighborhood sizes.

### Dataset reading

In [ ]:
vegetation = pd.DataFrame(read_las(DATADIR / "input" / VEGETATION_FILE), columns=COORDS + COULEURS)
falaise = pd.DataFrame(read_las(DATADIR / "input" / FALAISE_FILE), columns=COORDS + COULEURS)
eboulis = pd.DataFrame(read_las(DATADIR / "input" / EBOULIS_FILE), columns=COORDS + COULEURS)

In [ ]:
print("Dimension of `vegetation` dataset: ", vegetation.shape)
print("dimension of `falaise` dataset: ", falaise.shape)
print("dimension of `eboulis` dataset: ", eboulis.shape)

### Distance assessment with respect to neighborhood size

Let's consider the following neighborhood range:

In [ ]:
VOISINS_NUM = [10, 20, 30, 50, 100, 200, 500, 1000, 1500, 2000]

We can have an insight about the neighboring sphere dimension by computing the mean distance (in meter) for each neighborhood size, *e.g.* by considering 20-point samples.

In [ ]:
print("Vegetation neighborhood sizes:")
veg_sample = vegetation[COORDS].sample(20).values
gen = (request_tree(veg_sample, tree, num) for num in VOISINS_NUM)
d = [{"mean (in meter)": d.mean(axis=1)[0], "max (in meter)": d.max(axis=1)[0]} for d, _ in gen]
pd.DataFrame(d, index=VOISINS_NUM).T

In [ ]:
print("Cliff neighborhood sizes:")
falaise_sample = falaise[COORDS].sample(20).values
gen = (request_tree(falaise_sample, tree, num) for num in VOISINS_NUM)
d = [{"mean (in meter)": d.mean(axis=1)[0], "max (in meter)": d.max(axis=1)[0]} for d, _ in gen]
pd.DataFrame(d, index=VOISINS_NUM).T

With 1000 neighbors, the neighboring sphere is close to 1 meter.

### Feature study

We will compare some feature profile with respect to neighboring sphere size (resp. neighbor quantity), for each class (`falaise`, `eboulis`, `vegetation`).

In [ ]:
FEATURE_PATH = Path("../..", DATAFOLDER, "output", PREFIX_NAME, "features")

**Disclaimer** The next cell must be run *after* running the featurization step for each scene sample.

Consider the following command, for each class:

```
geo3d featurize -d <datafolder> -i Pombourg_<class>.las -c r g b -t 500 -n 10 20 30 50 100 200 500 1000 1500 2000 --label-scene
```

where `datafolder` is the name of the data folder at the project root, on your file system, and `class` is the requested class (`vegetation`, `eboulis`, `falaise`).

In [ ]:
vegetation_store = pd.HDFStore(FEATURE_PATH / "features_vegetation.h5", "r")
falaise_store = pd.HDFStore(FEATURE_PATH / "features_falaise.h5", "r")
eboulis_store = pd.HDFStore(FEATURE_PATH / "features_eboulis.h5", "r")

In [ ]:
KEYS = {i: "/num_{:04d}".format(i) for i in VOISINS_NUM}

In [ ]:
def mean_features(store, feature_name):
    result = pd.Series({num: store["/num_{:04d}".format(num)][feature_name].mean() for num in VOISINS_NUM})
    result.name = feature_name
    return result

#### Distance : radius & radius 2D

In [ ]:
radius = pd.DataFrame({"vegetation": mean_features(vegetation_store, "radius"),
                       "falaise": mean_features(falaise_store, "radius"),
                       "eboulis": mean_features(eboulis_store, "radius")})

In [ ]:
radius

In [ ]:
radius.plot(kind="bar", title="radius")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean radius (m)")

In [ ]:
radius_2d = pd.DataFrame({"vegetation": mean_features(vegetation_store, "radius_2D"),
                         "falaise": mean_features(falaise_store, "radius_2D"),
                         "eboulis": mean_features(eboulis_store, "radius_2D")})

In [ ]:
radius_2d.plot(kind="bar", title="Radius 2D (x, y)")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean 2D radius (m)")

#### Densité - 2D et 3D

In [ ]:
key = "density_2D"
density_2d = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                           "falaise": mean_features(falaise_store, key),
                           "eboulis": mean_features(eboulis_store, key)})

In [ ]:
density_2d.plot(kind="bar", title="Density 2D")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean 2D density (m)")

In [ ]:
key = "density"
density_3d = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                           "falaise": mean_features(falaise_store, key),
                           "eboulis": mean_features(eboulis_store, key)})

In [ ]:
density_3d.plot(kind="bar", title="Density 3D")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean density (m)")

#### Autres features 2D : eigen sum & ratio

In [ ]:
key = "eigenvalue_ratio_2D"
eigenratio_2d = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                           "falaise": mean_features(falaise_store, key),
                           "eboulis": mean_features(eboulis_store, key)})

In [ ]:
eigenratio_2d.plot(kind="bar", title="Eigen ratio 2D")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean e2 / e1")

In [ ]:
key = "eigenvalue_sum_2D"
eigensum_2d = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                           "falaise": mean_features(falaise_store, key),
                           "eboulis": mean_features(eboulis_store, key)})

In [ ]:
eigensum_2d.plot(kind="bar", title="Eigen sum 2D")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean eigenvalue sum")

#### alpha & beta

In [ ]:
alpha = pd.DataFrame({"vegetation": mean_features(vegetation_store, "alpha"),
                     "falaise": mean_features(falaise_store, "alpha"),
                     "eboulis": mean_features(eboulis_store, "alpha")})

The bigger the `alpha`, the more 1D the dataset.

In [ ]:
alpha.plot(kind='bar', title="alpha")
plt.xlabel("Neighborhood size")
plt.ylabel("alpha")

In [ ]:
beta = pd.DataFrame({"vegetation": mean_features(vegetation_store, "beta"),
                     "falaise": mean_features(falaise_store, "beta"),
                     "eboulis": mean_features(eboulis_store, "beta")})

The bigger `beta` is, the more 2D the dataset.

In [ ]:
beta.plot(kind="bar", title="beta")
plt.xlabel("Neighborhood size")
plt.ylabel("beta")

Then, the bigger `1 - (alpha + beta)` is, the more 3D the dataset.

In [ ]:
(1 - (alpha + beta)).plot(kind="bar", title='1 - (alpha + beta)')
plt.xlabel("Neighborhood size")
plt.ylabel("1 - (alpha + beta)")

We want to plot the evolution of these features for each class, with respect to neighborhood size.

In [ ]:
triangle_vegetation = pd.DataFrame({"alpha": mean_features(vegetation_store, "alpha"),
                     "beta": mean_features(vegetation_store, "beta")})

triangle_vegetation["gamma"] = triangle_vegetation.eval("1 - alpha - beta")

In [ ]:
triangle_vegetation

In [ ]:
triangle_vegetation.plot()

In [ ]:
triangle_falaise = pd.DataFrame({"alpha": mean_features(falaise_store, "alpha"),
                     "beta": mean_features(falaise_store, "beta")})

triangle_falaise["gamma"] = triangle_falaise.eval("1 - alpha - beta")

In [ ]:
triangle_falaise.plot()

In [ ]:
triangle_eboulis = pd.DataFrame({"alpha": mean_features(eboulis_store, "alpha"),
                     "beta": mean_features(eboulis_store, "beta")})

triangle_eboulis["gamma"] = triangle_eboulis.eval("1 - alpha - beta")

In [ ]:
triangle_eboulis.plot()

#### Verticality coefficient

In [ ]:
key = "verticality"
verticality = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                           "falaise": mean_features(falaise_store, key),
                           "eboulis": mean_features(eboulis_store, key)})

In [ ]:
verticality.plot(kind="bar", title="Verticality")
plt.xlabel("Neighborhood size")
plt.ylabel("Verticality")

#### Linerarity, planarity & scattering

In [ ]:
key = "linearity"
linearity = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                          "falaise": mean_features(falaise_store, key),
                          "eboulis": mean_features(eboulis_store, key)})

In [ ]:
linearity.plot(kind="bar", title="linearity")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean (e1-e2)/e1")

In [ ]:
key = "planarity"
planarity = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                          "falaise": mean_features(falaise_store, key),
                          "eboulis": mean_features(eboulis_store, key)})

In [ ]:
planarity.plot(kind="bar", title="planarity")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean (e2 - e3)/e1")

In [ ]:
key = "scattering"
scattering = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                          "falaise": mean_features(falaise_store, key),
                          "eboulis": mean_features(eboulis_store, key)})

In [ ]:
scattering.plot(kind="bar", title="scattering")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean e3 / e1")

#### Curvature Change

In [ ]:
key = "curvature_change"
curvature = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                          "falaise": mean_features(falaise_store, key),
                          "eboulis": mean_features(eboulis_store, key)})

In [ ]:
curvature.plot(kind="bar", title="Curvature change")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean e3 / (sum e_i)")

#### Omnivariance

In [ ]:
key = "omnivariance"
omnivariance = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                          "falaise": mean_features(falaise_store, key),
                          "eboulis": mean_features(eboulis_store, key)})

In [ ]:
omnivariance.plot(kind="bar", title="omnivariance")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean sqrt(e1*e2*e3)")

#### Anisotropy

In [ ]:
key = "anisotropy"
anisotropy = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                          "falaise": mean_features(falaise_store, key),
                          "eboulis": mean_features(eboulis_store, key)})

In [ ]:
anisotropy.plot(kind="bar", title="anisotopry")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean (e1 - e3)/e1")

#### Eigentropy

In [ ]:
key = "eigenentropy"
eigentropy = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                          "falaise": mean_features(falaise_store, key),
                          "eboulis": mean_features(eboulis_store, key)})

In [ ]:
eigentropy.plot(kind="bar", title="eigen entropy")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean -sum(e_i * ln e_i)")

#### Eigenvalues Sum

In [ ]:
key = "eigenvalue_sum"
eigensum = pd.DataFrame({"vegetation": mean_features(vegetation_store, key),
                          "falaise": mean_features(falaise_store, key),
                          "eboulis": mean_features(eboulis_store, key)})

In [ ]:
eigensum.plot(kind="bar", title="eigenvalues sum")
plt.xlabel("Neighborhood size")
plt.ylabel("Mean sum(lambda_i)")

### Feature correlation

After concatenating the dataframes of different neighborhood sizes, we can compute a correlation matrix to have an insight of mathematical relations between our variables.

In [ ]:
VOISIN_30 = "/num_0030"    # rayon sphère environ 20cm
VOISIN_1000 = "/num_1000"  # rayon sphère environ 1m

In [ ]:
FEATURE_NAMES =  ['alpha',
 'beta',
 'radius',
 'z_range',
 'std_dev',
 'density',
 'verticality',
 'curvature_change',
 'linearity',
 'planarity',
 'scattering',
 'omnivariance',
 'anisotropy',
 'eigenentropy',
 'eigenvalue_sum',
 'radius_2D',
 'density_2D',
 'eigenvalue_sum_2D',
 'eigenvalue_ratio_2D']

In [ ]:
def concat_features(voisin_num):
    return pd.concat([vegetation_store[voisin_num],
                      falaise_store[voisin_num],
                      eboulis_store[voisin_num]], axis=0, ignore_index=True)

In [ ]:
features_30 = concat_features(VOISIN_30)

In [ ]:
features_1000 = concat_features(VOISIN_1000)

In [ ]:
features_1000.columns

In [ ]:
rho_30 = features_30[FEATURE_NAMES].corr()

In [ ]:
rho_1000 = features_1000[FEATURE_NAMES].corr()

We can re-order the feature names.

In [ ]:
cols = pd.Index(['beta', 'planarity', 'anisotropy', 'density_2D', 'density',
       'verticality', 'eigenvalue_ratio_2D', 'eigenvalue_sum_2D',
       'eigenvalue_sum', 'radius_2D', 'radius', 'std_dev', 'z_range', 'alpha',
       'linearity', 'eigenentropy', 'omnivariance', 'scattering',
       'curvature_change'])

We have the following feature correlation matrix for 30 neighbors.

In [ ]:
with sns.axes_style("white", {'xtick.major.size': 12.0}):
    fig, ax = plt.subplots(figsize=(16, 10))
    sns.heatmap(rho_30.loc[cols, cols], square=True, cmap='RdBu_r',
                center=0., linewidths=.5, annot=True, fmt=".1f", ax=ax)

Then let's plot the same information for 1000 neighbors.

In [ ]:
with sns.axes_style("white", {'xtick.major.size': 12.0}):
    fig, ax = plt.subplots(figsize=(16, 10))
    sns.heatmap(rho_1000.loc[cols, cols], square=True, cmap='RdBu_r',
                center=0., linewidths=.5, annot=True, fmt=".1f", ax=ax)

### Running a PCA on geometric features

In order to run this additional analysis, we need a few more dependencies.

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
from geo3dfeatures.tools.kmean import load_features
from geo3dfeatures.features import max_normalize

As a first attempt to remove colinearities between the features in a generalisable way, we can apply Principle Component Analysis to the point cloud features for the whole scene, and for a small set of neighborhood sizes.

In [ ]:
scene_store = vegetation_store = pd.HDFStore(FEATURE_PATH / "features.h5", "r")

In [ ]:
SCENE_NEIGHBORHOODS = [50, 200, 1000]

In [ ]:
features = load_features(Path("..", "..", DATAFOLDER), "Pombourg", SCENE_NEIGHBORHOODS)

In [ ]:
features.columns

We consider a standardized version of the data.

In [ ]:
norm_features = StandardScaler().fit_transform(features.drop(columns=["x", "y"]))

Then we run a PCA with 10 components, by assuming that the whole information can be summarized in less dimensions.

In [ ]:
model = PCA(10).fit(norm_features)

As depicted by the following plot, the first component explain a large part of the model variance (~50%), whilst a projection on a 3 dimension space can explain 75% of the variance.

In [ ]:
with sns.axes_style("white", {'xtick.major.size': 12.0}):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.bar(range(model.n_components), model.explained_variance_)
    ax2 = ax.twinx()
    ax2.set_ylim(0, 1)
    ax2.plot(range(model.n_components), np.cumsum(model.explained_variance_ratio_), "r")
    ax2.hlines(y=np.linspace(0, 1, 11), xmin=0, xmax=10, color='gray')

To enhance the insight on the state of features, we can plot all of them in a big heatmap...

In [ ]:
pca_components = pd.DataFrame(model.components_.T, index=features.drop(columns=["x", "y"]).columns)

In [ ]:
with sns.axes_style("white", {'xtick.major.size': 12.0}):
    fig, ax = plt.subplots(figsize=(10, 20))
    sns.heatmap(pca_components, cmap='RdBu_r', fmt=".2f", ax=ax, center=0., linewidths=0.5, annot=True)

As a summary, we can see that the first component mixes a large part of the features, and the different neighborhood sizes do not really introduce variability. The second component brings a different information, based on verticality coefficients and 2D eigenvalue ratios. The third component is a slight variation of the first one. The fifth component is noticeable as it contains information about the color (with no particular distinction between RGB channels).

** **TDLR Interpretation through PCA can bring further insights on data, and confirm the information provided by correlation matrix. However a raw use of PCA components as explicative variables in a (un)supervised learning algorithm does not seem relevant.** **